<a href="https://colab.research.google.com/github/kimjaehwankimjaehwan/japan/blob/main/japanese_volcanoes_on_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install lets-plot --quiet
import lets_plot
lets_plot.__version__

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.4 MB/s eta 0:00:00


'4.4.0'

In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd

from lets_plot import *
from lets_plot import tilesets
LetsPlot.setup_html()
LetsPlot.set(tilesets.NASA_GREYSCALE_SHADED_RELIEF_30M)

## Volcanoes Coordinates

Volcano coordinates are given in DMS format and should be converted to decimal degrees for plotting on a map.

In [5]:
df_volc = pd.read_csv("/content/drive/MyDrive/일본/일본화산/Japan_Volcano.csv", encoding ='utf-8')
df_volc.head(3)

,Unnamed: 0,Name,Elevation_meters,Elevation_ft,Coordinates,Last_eruption,Region
0,0,Akaigawa Caldera,725,2379,﻿43.083°N 140.817°E﻿,1.3 Ma BP,Hokkaido
1,1,Mount Atosanupuri,512,1680,﻿43.610°N 144.438°E﻿,1000-200 BP,Hokkaido
2,2,Daisetsuzan Volcanic Group,2290,7513,﻿43.663°N 142.854°E﻿,AD 1739,Hokkaido


In [6]:
def dms_to_decimal(dms_string):
    """
    Convert DMS coordinates to decimal degrees.

    >>> dms_to_decimal("99.539°E")
    99.539

    >>> dms_to_decimal("99.539°W")
    -99.539

    """
    degrees, direction = dms_string.split('°')
    degrees = float(degrees)

    # Adjusting the sign based on direction
    if direction in ['S', 'W']:
        degrees *= -1

    return degrees


In [7]:
df_volc = df_volc.dropna(subset=['Coordinates'])

# Split 'Coordinates' DMS str.
lat_lon_dms = df_volc.Coordinates.str.split(' ').apply(lambda lst: lst[1:3])

# Remove BOM symbols (\ufeff) and
# convert DMS strings to decimal degrees.
lat_lon_dd = lat_lon_dms.apply(lambda lst: [dms_to_decimal(v.replace('\ufeff', '')) for v in lst])

# Create a GeoDataframe by adding the 'geometry' column.
latitudes = lat_lon_dd.apply(lambda pair: pair[0])
longitudes = lat_lon_dd.apply(lambda pair: pair[1])

gdf_volc = gpd.GeoDataFrame(df_volc, geometry=gpd.points_from_xy(longitudes, latitudes))
gdf_volc.head(3)

,Unnamed: 0,Name,Elevation_meters,Elevation_ft,Coordinates,Last_eruption,Region,geometry
0,0,Akaigawa Caldera,725,2379,﻿43.083°N 140.817°E﻿,1.3 Ma BP,Hokkaido,POINT (140.81700 43.08300)
1,1,Mount Atosanupuri,512,1680,﻿43.610°N 144.438°E﻿,1000-200 BP,Hokkaido,POINT (144.43800 43.61000)
2,2,Daisetsuzan Volcanic Group,2290,7513,﻿43.663°N 142.854°E﻿,AD 1739,Hokkaido,POINT (142.85400 43.66300)


## Volcanoes on Map

In [8]:
nice_view_options = dict(
    location = [129.450953, 30.151413, 144.875758, 42.515155],
    zoom = 6
)

In [9]:
(ggplot() + geom_livemap(**nice_view_options, const_size_zoomin=2) +
 geom_point(data=gdf_volc,
            tooltips=layer_tooltips(["Region", "Last_eruption"])
                .title("@Name")
                .line("Elevation | @Elevation_meters(m)/@Elevation_ft(ft)"),
            fill='#75DFBD', color='white', shape=24, size=8, alpha=.7) +
 ggsize(1000, 1000)
)

## Volcanoes Heatmap

In [10]:
coords_data = dict(
    lon=longitudes,
    lat=latitudes
)

(ggplot(coords_data) + geom_livemap() +
 # Volcano locations
 geom_point(data=gdf_volc,
            tooltips=layer_tooltips(["Region", "Last_eruption"])
                .title("@Name")
                .line("Elevation | @Elevation_meters(m)/@Elevation_ft(ft)"),
            color='white', size=2) +

 # Heatmap by Count-2D stat.
 geom_bin2d(aes("lon", "lat"), bins=[10, 15], alpha=.6, tooltips='none') +

 scale_fill_viridis(option='magma', guide=guide_colorbar(barwidth=4, barheight=300)) +
 flavor_high_contrast_dark() +
 theme(plot_background=element_rect(fill='#585858')) +
 theme(legend_position=[1, 0.5], legend_justification=[1, 0.5], legend_background=element_rect(fill='#585858'),
       plot_inset=0) +
 ggsize(1000, 1000)
)

## Heatmap and Contours of a 2D Density

In [11]:
(ggplot(coords_data) + geom_livemap() +
 # Volcano locations
 geom_point(data=gdf_volc,
            tooltips=layer_tooltips(["Region", "Last_eruption"])
                .title("@Name")
                .line("Elevation | @Elevation_meters(m)/@Elevation_ft(ft)"),
            color='white', size=2) +

 # Heatmap by Count-2D stat.
 geom_bin2d(aes("lon", "lat"), bins=[10, 15], alpha=.6, tooltips='none') +
 scale_fill_viridis(option='magma', guide=guide_colorbar(barwidth=4, barheight=300)) +

 # Contours by Density-2D stat.
 geom_density2d(aes("lon", "lat", color='..level..'), size=0.4, alpha=.6, tooltips='none', show_legend=False) +
 scale_color_viridis(option='turbo') +

 flavor_high_contrast_dark() +
 theme(plot_background=element_rect(fill='#585858')) +
 theme(legend_position=[1, 0.5], legend_justification=[1, 0.5], legend_background=element_rect(fill='#585858'),
       plot_inset=0) +
 ggsize(1000, 1000)
)

## Filled Contour Bands

In [12]:
(ggplot(coords_data) + geom_livemap() +
 # Volcano locations
 geom_point(data=gdf_volc,
            tooltips=layer_tooltips(["Region", "Last_eruption"])
                .title("@Name")
                .line("Elevation | @Elevation_meters(m)/@Elevation_ft(ft)"),
            color='white', size=2) +

 # Contours by Density-2D Filled stat.
 geom_density2df(aes("lon", "lat", fill='..level..'), alpha=.6, tooltips='none', show_legend=False) +
 scale_fill_viridis(option='magma') +

 flavor_high_contrast_dark() +
 theme(plot_background=element_rect(fill='#585858'),
       plot_inset=0) +
 ggsize(1000, 1000)
)